In [1]:
import torch
from simulator import blogm, bSqc, Neg, Sa
%run 'utils.py'

## load test result

In [7]:
# MPO no ancilla

seed = 0

mps_out = []
for N in range(4,18,2):
    l = torch.load(f'MPO/seed{seed}/record/mpo_N={N}_pa.pt')
    mps_out.append([N, torch.tensor(l['test Sqc']).view(-1,200).mean(-1)[-1].item(), torch.tensor(l['test Neg']).view(-1,200).mean(-1)[-1].item(), torch.tensor(l['test Sa']).view(-1,200).mean(-1)[-1].item()])
mps_out

[[4, 0.37293559312820435, 0.4144977629184723, 0.6910765767097473],
 [6, 0.4041719138622284, 0.40549445152282715, 0.6902521252632141],
 [8, 0.4909408688545227, 0.3823964595794678, 0.694428026676178],
 [10, 0.4945811927318573, 0.37955597043037415, 0.6941173076629639],
 [12, 0.5601881146430969, 0.3584675192832947, 0.6951113939285278],
 [14, 0.6608949899673462, 0.32349127531051636, 0.6954890489578247],
 [16, 0.6998947262763977, 0.30740034580230713, 0.6948459148406982]]

In [ ]:
torch.save(mps_out, 'mps_out_a.pt')

In [8]:
# GPT ancilla

seed = 0

gpt_out = []
for N in range(4,18,2):
    l = torch.load(f'GPT_together/seed{seed}/record/gpt_N={N}_pa.pt')
    gpt_out.append([N, torch.tensor(l['test Sqc']).view(-1,200).mean(-1)[-1].item(), torch.tensor(l['test Neg']).view(-1,200).mean(-1)[-1].item(), torch.tensor(l['test Sa']).view(-1,200).mean(-1)[-1].item()])
gpt_out

[[4, 0.36865946650505066, 0.4146355390548706, 0.6908187866210938],
 [6, 0.4026847779750824, 0.4054212272167206, 0.690269947052002],
 [8, 0.4854764938354492, 0.3825090825557709, 0.69195955991745],
 [10, 0.49391019344329834, 0.37696030735969543, 0.6919362545013428],
 [12, 0.5604791045188904, 0.3561559021472931, 0.6923449039459229],
 [14, 0.6588576436042786, 0.32264041900634766, 0.691292941570282],
 [16, 0.7095317840576172, 0.3043021857738495, 0.6920678615570068]]

In [ ]:
torch.save(gpt_out, 'gpt_out_a.pt')

In [4]:
# post select

seed = 0
train_size = 500000

post_out = []
for N in range(4,18,2):
    torch.manual_seed(seed)
    prepseq, shadow_state, rhoS = torch_data(f'data/post_selected/data_{N}pa.pickle', shuffle=True)
    prepseq, shadow_state, rhoS = prepseq[:600000], shadow_state[:600000], rhoS[:600000]
    prepseq, shadow_state, rhoS = prepseq[train_size:], shadow_state[train_size:], rhoS[train_size:]
    for i in range(2):
        for j in range(2):
            idx = (prepseq[:,::2].sum(-1)%2 == i)*(prepseq[:,1::2].sum(-1)%2 == j)
            rhoC = rhoS[idx].mean(0, keepdim=True)
            post_out.append([N, bSqc(rhoC,rhoC).item(), Neg(rhoC, rhoC).item(), Sa(rhoC, rhoC).item()])
post_out = torch.tensor(post_out).view(-1, 4, 4).mean(1).tolist()
post_out

[[4.0, 0.3419240713119507, 0.41506320238113403, 0.6903237104415894],
 [6.0, 0.3942759037017822, 0.4043729603290558, 0.6904787421226501],
 [8.0, 0.4830217957496643, 0.382238507270813, 0.6929681301116943],
 [10.0, 0.4859117269515991, 0.3796651363372803, 0.6924377679824829],
 [12.0, 0.5496563911437988, 0.357470840215683, 0.6929038763046265],
 [14.0, 0.6560057997703552, 0.32149142026901245, 0.6927497386932373],
 [16.0, 0.6965165138244629, 0.3039205074310303, 0.6918038129806519]]

In [5]:
torch.save(post_out, 'post_out_a.pt')

## prepare N-mixed data

In [19]:
seed = 0
prepseq_all, shadow_state_all, rhoS_all = [], [], []
for N in range(4,18,2):
    torch.manual_seed(seed)
    prepseq, shadow_state, rhoS = torch_data(f'data/post_selected/data_{N}pa.pickle', shuffle=True)
    
    prepseq, shadow_state, rhoS = prepseq[:600000], shadow_state[:600000], rhoS[:600000]
    
    prepseq = torch.cat([prepseq+2, torch.zeros(prepseq.shape[0], 14-prepseq.shape[1], dtype=prepseq.dtype), torch.ones(prepseq.shape[0], 1, dtype=prepseq.dtype)], 1)
    prepseq_all.append(prepseq[None,...])
    shadow_state_all.append(shadow_state[None,...])
    rhoS_all.append(rhoS[None,...])
prepseq_all = torch.cat(prepseq_all)
shadow_state_all = torch.cat(shadow_state_all)
rhoS_all = torch.cat(rhoS_all)

In [20]:
prepseq_all.shape

torch.Size([7, 600000, 15])

In [21]:
torch.save(prepseq_all, f'data/post_selected/prepseq_all.pt')
torch.save(shadow_state_all, f'data/post_selected/shadow_state_all.pt')
torch.save(rhoS_all, f'data/post_selected/rhoS_all.pt')

In [22]:
prepseq_all = torch.load(f'data/post_selected/prepseq_all.pt')
shadow_state_all = torch.load(f'data/post_selected/shadow_state_all.pt')
rhoS_all = torch.load(f'data/post_selected/rhoS_all.pt')

In [23]:
train_ratio = 5/6

train_size = int(prepseq_all.shape[1]*train_ratio)
test_size = prepseq_all.shape[1]-train_size

prepseq_train, prepseq_test = prepseq_all[:,:train_size], prepseq_all[:,train_size:]
shadow_state_train, shadow_state_test = shadow_state_all[:,:train_size], shadow_state_all[:,train_size:]
rhoS_train, rhoS_test = rhoS_all[:,:train_size], rhoS_all[:,train_size:]

In [24]:
seed = 0
torch.manual_seed(seed)
prepseq_train, shadow_state_train, rhoS_train = shuffle(prepseq_train.flatten(0,1), shadow_state_train.flatten(0,1), rhoS_train.flatten(0,1))

In [25]:
torch.save(prepseq_train, f'data/post_selected/prepseq_train.pt')
torch.save(shadow_state_train, f'data/post_selected/shadow_state_train.pt')
torch.save(rhoS_train, f'data/post_selected/rhoS_train.pt')

In [26]:
torch.save(prepseq_test, f'data/post_selected/prepseq_test.pt')
torch.save(shadow_state_test, f'data/post_selected/shadow_state_test.pt')
torch.save(rhoS_test, f'data/post_selected/rhoS_test.pt')